# Mp 1 0 2_Read Ntuples Fill Histos And Fit
Read n-tuples in distinct workers, fill histograms, merge them and fit.
We express parallelism with multiprocessing as it is done with multithreading
in mt102_readNtuplesFillHistosAndFit.




**Author:** Danilo Piparo  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:16 AM.</small></i>

No nuisance for batch execution

In [1]:
gROOT->SetBatch();

---------------------------------------
 Perform the operation sequentially

In [2]:
TChain inputChain("multiCore");
inputChain.Add("mp101_multiCore_*.root");
if (inputChain.GetNtrees() <= 0) {
   Printf(" No files in the TChain: did you run mp101_fillNtuples.C before?");
   return 1;
}
TH1F outHisto("outHisto", "Random Numbers", 128, -4, 4);
inputChain.Draw("r >> outHisto");
outHisto.Fit("gaus");

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


 FCN=135.105 FROM MIGRAD    STATUS=CONVERGED      57 CALLS          58 TOTAL
                     EDM=2.49215e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.98665e+05   1.36722e+02   6.35090e-01   8.34171e-09
   2  Mean         5.64893e-05   2.23739e-04   1.27452e-06   9.95505e-02
   3  Sigma        1.00002e+00   1.58821e-04   4.42085e-07  -9.89135e-03


---------------------------------------
 We now go MP!
 TProcessExecutor offers an interface to directly process trees and chains without
 the need for the user to go through the low level implementation of a
 map-reduce.

We adapt our parallelisation to the number of input files

In [3]:
const auto nFiles = inputChain.GetListOfFiles()->GetEntries();

This is the function invoked during the processing of the trees.

In [4]:
auto workItem = [](TTreeReader &reader) {
   TTreeReaderValue<Float_t> randomRV(reader, "r");
   auto partialHisto = new TH1F("outHistoMP", "Random Numbers", 128, -4, 4);
   while (reader.Next()) {
      partialHisto->Fill(*randomRV);
   }
   return partialHisto;
};

Create the pool of processes

In [5]:
ROOT::TTreeProcessorMP workers(nFiles);

Process the tchain

In [6]:
auto sumHistogram = workers.Process(inputChain, workItem, "multiCore");
sumHistogram->Fit("gaus", 0);

return 0;

 FCN=135.105 FROM MIGRAD    STATUS=CONVERGED      51 CALLS          52 TOTAL
                     EDM=2.45494e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.98665e+05   1.36722e+02   6.35090e-01   1.50931e-08
   2  Mean         5.64893e-05   2.23739e-04   1.27452e-06   9.86882e-02
   3  Sigma        1.00002e+00   1.58821e-04   4.42085e-07   8.26031e-03


Draw all canvases 

In [7]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()